In [1]:
import os
import pyxdf # xdf file importation
import numpy as np
from my_functions import *

### Ensure that the root directory is the project directory
In some cases, the root directory may not be the project directory. This is usually the case when the notebook is run from a subdirectory of the project directory.

In [2]:
# if pwd is SRC  change to root
print("Current working directory: ", os.getcwd())
if os.path.basename(os.getcwd()) == "SRC":
    os.chdir("..")
print("Current working directory: ", os.getcwd())
print(os.path.basename(os.getcwd()))

Current working directory:  c:\Users\iWiss\Documents\GitHub\Stage_Signal_Physio\SRC
Current working directory:  c:\Users\iWiss\Documents\GitHub\Stage_Signal_Physio
Stage_Signal_Physio



## Initialization

### File selection and loading

In [3]:
# Define the xdf file path
FILENAME = "001_MolLud_20201112_1_c.xdf"
# FILENAME="020_DesMar_20211129_1_c.xdf"
# path=os.path.normpath("../DAT/Input/001_MolLud_20201112_1_c.xdf")
path = os.path.normpath("DAT/INPUT/"+FILENAME)
print("path: ",path)

path:  DAT\INPUT\001_MolLud_20201112_1_c.xdf


### Loading streams of interest

#### Storing the data of interest in two variables (markers and signal) 

In [4]:
# load_xdf returns selected streams as list of streams (ie.dictionary)
EEG_Stream, EEG_fileheader = pyxdf.load_xdf(path, select_streams=[{'type': 'EEG'}])
Mouse_markers_Stream, Mouse_markers_header = pyxdf.load_xdf(path, select_streams=[{'type': 'Markers', 'name': 'MouseToNIC'}])

if len(EEG_Stream) and len(Mouse_markers_Stream) ==1:
    EEG_Stream=EEG_Stream[-1] #acess to the only EEG stream of the list
    Mouse_markers_Stream=Mouse_markers_Stream[-1]
else:
    print("Multiple streams matching type restriction")


#### Defining the channels names

In [5]:
# definition of the EEG channels' names
channels_dict = {"Channel_1": "C4",
                "Channel_2": "FC2",
                "Channel_3": "FC6",
                "Channel_4": "CP2",
                "Channel_5": "C3",
                "Channel_6": "FC1",
                "Channel_7": "FC5",
                "Channel_8": "CP1"}

### Shaping recoding parameters' data

In [6]:
Srate = EEG_Stream["info"]["effective_srate"]
# Markers
# Markers start with a time relative to the execution time of the recording
Marker_times = (
    Mouse_markers_Stream["time_stamps"]-EEG_Stream["time_stamps"][0])

# select the marker labels
Markers_labels = Mouse_markers_Stream["time_series"]

# Creation of a 2D array Markers_times_labels: [[markers_timesstamps],[markers_labels]]
Markers_times_labels = np.column_stack((Marker_times, Markers_labels))

In [7]:
# EEG
# format eeg times_stamps in seconds relative to the execution time of the recording
EEG_times = EEG_Stream["time_stamps"]-EEG_Stream["time_stamps"][0]
# Amplitude of voltage recorded by each electrode of the recording set-up
EEG_raw_amplitudes = EEG_Stream["time_series"]

### Export

#### Data to be exported

In [8]:
print("Srate:",Srate)
print("EEG_times type",type(EEG_times)," shape",EEG_times.shape)
print("EEG_raw_amplitudes type",type(EEG_raw_amplitudes), "shape",EEG_raw_amplitudes.shape)

times_amplitudes=np.column_stack((EEG_raw_amplitudes,EEG_times))

print(times_amplitudes[:,-1]==EEG_times)


Srate: 499.9977270435165
EEG_times type <class 'numpy.ndarray'>  shape (221938,)
EEG_raw_amplitudes type <class 'numpy.ndarray'> shape (221938, 8)
[ True  True  True ...  True  True  True]


#### Export to CSV file

In [9]:
#creating filename
originalfilename=FILENAME.replace(".xdf","_")

exportfilename=originalfilename+str(round(Srate,3))+"Hz.csv"

exportfilepath=os.path.normpath("DAT/INPUT/"+exportfilename)
print(exportfilepath)

DAT\INPUT\001_MolLud_20201112_1_c_499.998Hz.csv


In [10]:
header = ', '.join([f"{key}:{value}" for key, value in channels_dict.items()])
print(header)
header=header+',time(s)'
print(header)


Channel_1:C4, Channel_2:FC2, Channel_3:FC6, Channel_4:CP2, Channel_5:C3, Channel_6:FC1, Channel_7:FC5, Channel_8:CP1
Channel_1:C4, Channel_2:FC2, Channel_3:FC6, Channel_4:CP2, Channel_5:C3, Channel_6:FC1, Channel_7:FC5, Channel_8:CP1,time(s)


In [11]:
#np.savetxt(exportfilepath,times_amplitudes,delimiter=",")
np.savetxt(exportfilepath, times_amplitudes, delimiter=',', header=header, comments='', fmt='%d')